# 🌾 AgriBot NER Training with PhoBERT (IoT Focus)

Train PhoBERT-based NER model for Vietnamese Agricultural IoT Chatbot

**Entity Types (6 types):**
- `DATE`: tháng này, quý 1, năm nay, tháng 11, ...
- `CROP`: cam sành, lúa ST25, xoài cát chu, ...
- `AREA`: khu A, khu B, khu 1, ...
- `DURATION`: 5 phút, 10 phút, 1 giờ, ...
- `DEVICE`: máy bơm, đèn, tưới, bơm, ...
- `METRIC`: nhiệt độ, độ ẩm, ánh sáng, ...

## 📦 Step 1: Install Dependencies

In [ ]:
!pip install transformers datasets torch scikit-learn seqeval pandas -q

## 📊 Step 2: Load Training Data

**Upload CSV file** generated by `generate_ner_data_v2.py`

In [ ]:
import json
import pandas as pd
from typing import List, Dict
from google.colab import files

"""
chuyển định dạng data sang
 1. Text: ngô ngọt bị bệnh gì
   Entities: [(0, 8, 'CROP')]
"""

# Upload CSV file
print("📤 Please upload your CSV file (generated by generate_ner_data_v2.py)")
uploaded = files.upload()

# Get the uploaded filename
csv_filename = list(uploaded.keys())[0]
print(f"\n📂 Loading data from {csv_filename}...")

# Load CSV
df = pd.read_csv(csv_filename)

# Convert to training format: [(text, [(start, end, label), ...]), ...]
training_data = []

for _, row in df.iterrows():
    text = row['text']
    entities_json = json.loads(row['entities'])
    # Convert to (start, end, type) tuples
    entities = [(e['start'], e['end'], e['type']) for e in entities_json]
    training_data.append((text, entities))

print(f"\n✅ Loaded {len(training_data)} training examples from CSV")

# Show sample
print("\n📝 Sample data:")
for i, (text, entities) in enumerate(training_data[:5]):
    print(f"\n{i+1}. Text: {text}")
    print(f"   Entities: {entities}")
"""
📝 Sample data:
1. Text: ngô ngọt bị bệnh gì
   Entities: [(0, 8, 'CROP')]
2. Text: Thông tin về giống hành tím
   Entities: [(19, 27, 'CROP')]
3. Text: Sâu đục thân ở mướp xử lý như thế nào
   Entities: [(15, 19, 'CROP')]
4. Text: Cách trồng dưa lưới
   Entities: [(11, 19, 'CROP')]
5. Text: Tôi muốn biết về giống cam Cao Phong
   Entities: [(23, 36, 'CROP')]
"""

## 🔄 Step 3: Convert to BIO Format

In [ ]:
def convert_to_bio_format(data: List[tuple]) -> List[Dict]:
    """
    Tức là đầu vào là
    1. Text: ngô ngọt bị bệnh gì
    Entities: [(0, 8, 'CROP')]
    || Entity đang đc gán theo vị trí ký tự , ko phải theo word
    =>
    Đầu ra 
#   {
#     "tokens": ["ngô", "ngọt", "bị", "bệnh", "gì"],
#     "ner_tags": ["B-CROP", "I-CROP", "O", "O", "O"]
#   }

B1. Duyệt từng mẫu dữ liệu
B2. Tokenizen câu bằng split() ->   tokens = ["ngô", "ngọt", "bị", "bệnh", "gì"]
                                    labels = ["B-CROP", "I-CROP", "O", "O", "O"]
B3. Tạo mapping char index -> word index
    """


    """
    Convert annotated data to BIO format
    Chuyển data từ format vị trí ký tự sang format BIO
    Data gốc đánh dấu vị trí ký tự(start,end)
    AI cần nhãn cho từng từ (BIO tags)
text = "ngô ngọt bị bệnh gì"
entities = [(0, 8, 'CROP')]
tokens = ["ngô", "ngọt", "bị", "bệnh", "gì"]
labels = ["B-CROP", "I-CROP", "O", "O", "O"]

    """
    bio_data = []
    # duyệt từng mẫu, lấy text, entities từng mẫu
    for text, entities in data:
        # Tokenize by word
        words = text.split()
        labels = ['O'] * len(words)
# text = "ngô ngọt bị bệnh gì"
# words = ["ngô", "ngọt", "bị", "bệnh", "gì"]
# labels = ["O", "O", "O", "O", "O"]
        
        """
        tạo bản đồ ký tự -> từ
        """
        # Create character to word index mapping
        char_to_word = {}
        current_pos = 0
        # FOR 1 -  CHO WORD 
        for word_idx, word in enumerate(words):
            # word_idx=0, word="ngô"
            # word_idx=1, word="ngọt"
            # word_idx=2, word="bị"
            word_start = text.find(word, current_pos)
            word_end = word_start + len(word)
            """
            ví dụ "ngô" range(0,3) = [0,1,2]
            char_to_word[0] = 0  # Ký tự 'n' thuộc từ 0 (idx) là ngô
            char_to_word[1] = 0  # Ký tự 'g' thuộc từ 0
            char_to_word[2] = 0  # Ký tự 'ô' thuộc từ 0

            ví dụ "ngọt" range(4, 8) = [4, 5, 6, 7]
            char_to_word[4] = 1  # Ký tự 'n' thuộc từ 1
            char_to_word[5] = 1  # Ký tự 'g' thuộc từ 1
            char_to_word[6] = 1  # Ký tự 'ọ' thuộc từ 1
            char_to_word[7] = 1  # Ký tự 't' thuộc từ 1
            """
            # char_idx 0 -> 3
            for char_idx in range(word_start, word_end):
                # n = 0
                # g = 0
                # ô = 0
                char_to_word[char_idx] = word_idx
            current_pos = word_end

            """
             VÍ DỤ HOÀN CHỈNH
Input:
text = "Bật tưới khu A trong 5 phút"
entities = [
    (4, 8, 'DEVICE'),    # "tưới"
    (9, 14, 'AREA'),     # "khu A"
    (21, 28, 'DURATION') # "5 phút"
]
Bước 1: Tách từ
words = ["Bật", "tưới", "khu", "A", "trong", "5", "phút"]
labels = ["O", "O", "O", "O", "O", "O", "O"]
Bước 2: Tạo bản đồ
text = "Bật tưới khu A trong 5 phút"
       0123456789...
char_to_word = {
    0: 0,  # 'B' → từ 0 ("Bật")
    1: 0,  # 'ậ' → từ 0
    2: 0,  # 't' → từ 0
    4: 1,  # 't' → từ 1 ("tưới")
    5: 1,  # 'ư' → từ 1
    6: 1,  # 'ớ' → từ 1
    7: 1,  # 'i' → từ 1
    9: 2,  # 'k' → từ 2 ("khu")
    10: 2, # 'h' → từ 2
    11: 2, # 'u' → từ 2
    13: 3, # 'A' → từ 3 ("A")
    ...
}
Bước 3: Gán nhãn
Entity 1: (4, 8, 'DEVICE') → "tưới"

range(4, 8) = [4, 5, 6, 7]
entity_words = {1}  # Chỉ từ 1 ("tưới")
labels[1] = "B-DEVICE"
Kết quả: ["O", "B-DEVICE", "O", "O", "O", "O", "O"]
Entity 2: (9, 14, 'AREA') → "khu A"

range(9, 14) = [9, 10, 11, 12, 13]
entity_words = {2, 3}  # Từ 2 ("khu") và từ 3 ("A")
labels[2] = "B-AREA"
labels[3] = "I-AREA"
Kết quả: ["O", "B-DEVICE", "B-AREA", "I-AREA", "O", "O", "O"]
Entity 3: (21, 28, 'DURATION') → "5 phút"

range(21, 28) = [21, 22, 23, 24, 25, 26, 27]
entity_words = {5, 6}  # Từ 5 ("5") và từ 6 ("phút")
labels[5] = "B-DURATION"
labels[6] = "I-DURATION"
Kết quả: ["O", "B-DEVICE", "B-AREA", "I-AREA", "O", "B-DURATION", "I-DURATION"]
Output cuối cùng:
{
    "tokens": ["Bật", "tưới", "khu", "A", "trong", "5", "phút"],
    "ner_tags": ["O", "B-DEVICE", "B-AREA", "I-AREA", "O", "B-DURATION", "I-DURATION"]
}
    trong cái range của từ vd ngô idx=0 thì range(0,3) đánh =0 hết biểu thị đó là chỗ ngô
    range(4,8) của ngọt (idx=1) đánh =1 hết biểu thị đó là chỗ của ngọt  
    char_to_word = {
    0: 0,  # 'n' → từ 0 ("ngô")
    1: 0,  # 'g' → từ 0
    2: 0,  # 'ô' → từ 0
    3: 0,  # ' ' → KHÔNG GÁN (khoảng trắng)
    4: 1,  # 'n' → từ 1 ("ngọt")
    5: 1,  # 'g' → từ 1
    6: 1,  # 'ọ' → từ 1
    7: 1,  # 't' → từ 1
    8: 1,  # ' ' → KHÔNG GÁN
    9: 2,  # 'b' → từ 2 ("bị")
    10: 2, # 'ị' → từ 2
    ...
}
            """
        
        # Assign BIO labels
#           1. Text: ngô ngọt bị bệnh gì
#           Entities: [(0, 8, 'CROP')]
        # FOR 2 - CHO ENTITIES
        for start, end, entity_type in entities:
            # Find words that overlap with entity span
            entity_words = set()
            # range(0, 8) = [0, 1, 2, 3, 4, 5, 6, 7]
            for char_idx in range(start, end):
                # char_idx 0 -> 8
#      char_to_word =
#       {
#           0:0, 1:0, 2:0,           # ngô
#           4:1, 5:1, 6:1, 7:1,      # ngọt
#           9:2, 10:2,               # bị
#           12:3,13:3,14:3,15:3,     # bệnh
#           17:4,18:4                # gì
#       }
                if char_idx in char_to_word:
                    entity_words.add(char_to_word[char_idx])
                # 0 -> 8 là có 0,1
                # entity_words = {0,1}
            """
char_idx=0 → char_to_word[0]=0 → entity_words={0}
char_idx=1 → char_to_word[1]=0 → entity_words={0}
char_idx=2 → char_to_word[2]=0 → entity_words={0}
char_idx=3 → KHÔNG TỒN TẠI (khoảng trắng)
char_idx=4 → char_to_word[4]=1 → entity_words={0, 1}
char_idx=5 → char_to_word[5]=1 → entity_words={0, 1}
char_idx=6 → char_to_word[6]=1 → entity_words={0, 1}
char_idx=7 → char_to_word[7]=1 → entity_words={0, 1}
Kết quả: entity_words = {0, 1 ||can be 2,3,4,7,9,....}
Entity "ngô ngọt" (ký tự 0-8) bao gồm từ 0 ("ngô") và từ 1 ("ngọt")
            """
            entity_words = sorted(entity_words)
            if entity_words:
                # First word gets B- tag
                # những thằng 0 thì đánh là B-CROP
                labels[entity_words[0]] = f"B-{entity_type}"
                # Remaining words get I- tag
                # còn lại đánh I-CROP
                for word_idx in entity_words[1:]:
                    labels[word_idx] = f"I-{entity_type}"
#   BIO_DATA CÓ DẠNG NÀY:
#   {
#     "tokens": ["ngô", "ngọt", "bị", "bệnh", "gì"],
#     "ner_tags": ["B-CROP", "I-CROP", "O", "O", "O"]
#   }

        bio_data.append({
            "tokens": words,
            "ner_tags": labels
        })
    
    return bio_data

bio_dataset = convert_to_bio_format(training_data)

# Display first example
print("\n📝 Example BIO format:")
example = bio_dataset[0]
for token, tag in zip(example['tokens'], example['ner_tags']):
    print(f"{token:20} → {tag}")

print(f"\n✅ Converted {len(bio_dataset)} examples to BIO format")
"""
📝 Example BIO format:
ngô                  → B-CROP
ngọt                 → I-CROP
bị                   → O
bệnh                 → O
gì                   → O
✅ Converted 2000 examples to BIO format
"""

## 🏷️ Step 4: Create Label Mapping

In [ ]:
# Extract all unique labels
all_labels = set()
for example in bio_dataset:
    all_labels.update(example['ner_tags'])

# Sort labels (O first, then B- tags, then I- tags)
label_list = sorted(all_labels, key=lambda x: (x != 'O', x))

# Create label mappings
label2id = {label: idx for idx, label in enumerate(label_list)}
id2label = {idx: label for label, idx in label2id.items()}

print(f"\n🏷️ Total labels: {len(label_list)}")
print("\nLabel mapping:")
for label, idx in label2id.items():
    print(f"{idx:2d}: {label}")

# Save label mapping
label_mapping = {
    "label_to_id": label2id,
    "id_to_label": id2label,
    "entity_types": list(set([label.split('-')[1] for label in label_list if '-' in label]))
}

with open('label_mapping.json', 'w', encoding='utf-8') as f:
    json.dump(label_mapping, f, ensure_ascii=False, indent=2)
# 🏷️ Total labels: 13

# Label mapping:
#  0: O
#  1: B-AREA
#  2: B-CROP
#  3: B-DATE
#  4: B-DEVICE
#  5: B-DURATION
#  6: B-METRIC
#  7: I-AREA
#  8: I-CROP
#  9: I-DATE
# 10: I-DEVICE
# 11: I-DURATION
# 12: I-METRIC

print("\n✅ Saved label_mapping.json")

## 📚 Step 5: Prepare Dataset for Training

In [ ]:
from datasets import Dataset
from sklearn.model_selection import train_test_split

# Convert to HuggingFace Dataset format
def prepare_dataset(bio_data, label2id):
    dataset_dict = {
        "tokens": [], #danh sách các token
        "ner_tags": [] #danh sách các nhãn (số theo label2id)
    }
    
    for example in bio_data:
        dataset_dict["tokens"].append(example["tokens"])
        # Convert labels to IDs
        tag_ids = [label2id[tag] for tag in example["ner_tags"]]
        dataset_dict["ner_tags"].append(tag_ids)
    # chuyển dict thành dataset obj
    #dataset obj là định dạng mà thư viện Transformers yêu cầu
    return Dataset.from_dict(dataset_dict)
#   BIO_DATA CÓ DẠNG NÀY:
#   {
#     "tokens": ["ngô", "ngọt", "bị", "bệnh", "gì"],
#     "ner_tags": ["B-CROP", "I-CROP", "O", "O", "O"]
#   }
# Split train/validation (80/20)
train_data, val_data = train_test_split(bio_dataset, test_size=0.2, random_state=42)
# ví dụ với 3 mẫu 
# dataset_dict = {
#    "tokens": [
#         ["ngô", "ngọt", "bị", "bệnh", "gì"],
#         ["Bật", "tưới", "khu", "A"],
#         ["Chi", "phí", "tháng", "này"]
#     ],
#     "ner_tags": [
#         [1, 2, 0, 0, 0],           # B-CROP, I-CROP, O, O, O
#         [0, 3, 5, 6],              # O, B-DEVICE, B-AREA, I-AREA
#         [0, 0, 7, 8]               # O, O, B-DATE, I-DATE
#     ]
# }
train_dataset = prepare_dataset(train_data, label2id)
val_dataset = prepare_dataset(val_data, label2id)

print(f"\n📊 Dataset split:")
print(f"  Training: {len(train_dataset)} examples")
print(f"  Validation: {len(val_dataset)} examples")
"""
bio_dataset = [
    {
        "tokens": ["ngô", "ngọt", "bị", "bệnh", "gì"],
        "ner_tags": ["B-CROP", "I-CROP", "O", "O", "O"]
    },
    {
        "tokens": ["Bật", "tưới", "khu", "A"],
        "ner_tags": ["O", "B-DEVICE", "B-AREA", "I-AREA"]
    },
    # ... 1998 mẫu khác
]
# Mẫu 1:
tokens: ["ngô", "ngọt", "bị", "bệnh", "gì"]
ner_tags: ["B-CROP", "I-CROP", "O", "O", "O"]
→ tag_ids: [1, 2, 0, 0, 0]
# Mẫu 2:
tokens: ["Bật", "tưới", "khu", "A"]
ner_tags: ["O", "B-DEVICE", "B-AREA", "I-AREA"]
→ tag_ids: [0, 3, 5, 6]
"""

## 🤖 Step 6: Load PhoBERT Model

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

model_name = "vinai/phobert-base"
num_labels = len(label_list)

print(f"Loading {model_name}...")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print(f"✅ Model loaded on {device}")
print(f"   Number of labels: {num_labels}")

## 🔧 Step 7: Tokenize Dataset

In [ ]:
"""
PhoBERT tokenizer không support word_ids(), cần manual alignment
"""
"""
Input (word-level):
Từ:    ["máy", "bơm"]
Nhãn:  [B-DEVICE, I-DEVICE]
PhoBERT tokenize (subword-level):
Token: ["_máy", "_bơm"]
Nhãn:  [?, ?]  ← Cần gán nhãn

Cell 7 thực hiện 4 bước chính:"

Ghép từ thành câu hoàn chỉnh

Input: ["Bật", "máy", "bơm"]
Output: "Bật máy bơm"
Tokenization với PhoBERT

Sử dụng PhoBERT tokenizer
Tham số: max_length=128, padding="max_length"
Khởi tạo labels với -100

-100 = ignore index (bỏ qua khi tính loss)
Dùng cho special tokens và padding
Manual Alignment

Căn chỉnh nhãn từ word-level sang token-level

Với mỗi token:
  1. Bỏ qua nếu là special token (<s>, </s>, <pad>)
  2. Decode token thành text
  3. So khớp với từ hiện tại (word_idx)
  4. Nếu khớp:
     - Gán nhãn: labels[i] = ner_tags[word_idx]
     - Nếu hết từ: word_idx++
  5. Nếu không khớp:
     - Chuyển sang từ tiếp theo: word_idx++
     - Thử gán nhãn lại
"""

"""
Mục đích biến dữ liệu NER dạng word-level(BIO) thành subword-level để PhoBERT có thể train được
Đầu vào 
tokens   = ["ngô", "ngọt", "bị", "bệnh", "gì"]
ner_tags = ["B-CROP", "I-CROP", "O", "O", "O"]
PhoBERT ko làm việc vs word mà với subword token 
<s> _ngô _ngọt _bị _bệnh _gì </s> <pad> <pad> ... tức 1 word có thể bị tách thành nhiều subword -> cần nhân bản / căn chỉnh label cho đúng token
"""

"""
Đang có data dạng word-level 
ngô     → B-CROP
ngọt   → I-CROP
bị     → O
bệnh   → O
gì     → O

NHƯNG PhoBERT KHÔNG làm việc với từ
PhoBERT nhìn thấy subword token: <s>  _ngô  _ngọt  _bị  _bệnh  _gì  </s> ,
Model chỉ biết từng token,
không biết “ngô” là một từ hoàn chỉnh.

==> Nếu 1 từ bị tách thành 2 token thì label phải gán cho token nào ?

TEXT:
"ngô ngọt bị bệnh gì"

WORD LEVEL:
ngô     ngọt     bị     bệnh     gì
B-CROP I-CROP   O       O        O

SUBWORD LEVEL (PhoBERT):
<s> _ngô _ngọt _bị _b  _ệnh _gì </s>

LABEL PHẢI KHỚP:
-100 B-CROP I-CROP O   O   -100 -100

"""

def tokenize_and_align_labels(examples):
    """
    Tokenize text and align NER labels with subword tokens
    PhoBERT tokenizer doesn't support word_ids(), so manual alignment
    """
    # Định dạng chuẩn HF Trainer cho NER
    tokenized_inputs = {
        "input_ids": [],
        "attention_mask": [],
        "labels": []
    }
    # Duyệt từng sapmle
    for tokens, ner_tags in zip(examples["tokens"], examples["ner_tags"]):
        # Join tokens back to text vì PhoBERT cần câu hoàn chỉnh để tokenize
        text = " ".join(tokens)

        # Tokenize the full text
        encoding = tokenizer(
            text,
            truncation=True, #trunc nếu >128 token
            max_length=128,
            padding="max_length",
            return_tensors=None #trả về list python(ko phải tensor)
        )
#         Mảnh:    [<s>,  _ngô,  _ng,  ọt,  _bị,  </s>]
#         Số ID:   [0,    1234,  5678, 9012, 3456, 2]
        """
        encoding = {
        # mỗi só đại diện cho 1 token
            "input_ids": [0, 1234, 5678, 9012, 3456, 7890, 2, 1, 1, 1, ...],  # 128 số
            "attention_mask": [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ...]             # 128 số
        }   
        input_ids: Mỗi số đại diện cho 1 token
        0: <s> (bắt đầu câu)
        1234: "_ngô"
        5678: "_ng"
        9012: "ọt"
        2: </s> (kết thúc câu)
        1: <pad> (padding)
        attention_mask:
        1: Token thật
        0: Padding (bỏ qua)
        """

        
        # Get token IDs
        # output kiểu 
        #         [
        #           [0, 1234, 5678, 9012, 3456, 2, 1, 1, ...]
        #         ]
        token_ids = encoding["input_ids"]
        attention_mask = encoding["attention_mask"]

        # Initialize labels with -100 (ignore index)
        labels = [-100] * len(token_ids)

        # Manual alignment: match each word to its tokens
        current_pos = 0
        word_idx = 0
        # <s>, _ngô, _ngọt, _bị, _bệnh, _gì, </s>, <pad>, <pad>, ...
        for i, token_id in enumerate(token_ids):
            # Skip special tokens
            if token_id in [tokenizer.bos_token_id, tokenizer.eos_token_id, tokenizer.pad_token_id]:
                continue
      
            # Decode token
            token_text = tokenizer.decode([token_id], skip_special_tokens=True).strip()
            
            """
            text = "ngô ngọt bị"
            tokens = ["ngô", "ngọt", "bị"]
            ner_tags = [1, 2, 0]  # B-CROP, I-CROP, O
            # Sau khi tokenize:
            token_ids = [0, 1234, 5678, 9012, 3456, 2, 1, 1, ...]
                          ↑   ↑     ↑     ↑     ↑    ↑
                        <s> _ngô  _ng   ọt   _bị  </s> <pad>
            
            """
            # Remove PhoBERT underscore prefix
            token_clean = token_text.replace("_", " ").strip()

            if not token_clean:
                continue

            # Try to match this token to a word
            # như for mà word_idx++ , <len
            if word_idx < len(tokens): 
                word = tokens[word_idx]

                # Check if this token is part of the current word
                if token_clean.lower() in word.lower() or word.lower().startswith(token_clean.lower()):
                    # Assign the label for this word
                    labels[i] = ner_tags[word_idx]

                    # Check if we've finished this word
                    if token_clean.lower() == word.lower():
                        word_idx += 1
                else:
                    # Move to next word
                    word_idx += 1
                    if word_idx < len(tokens):
                        labels[i] = ner_tags[word_idx]

        tokenized_inputs["input_ids"].append(encoding["input_ids"])
        tokenized_inputs["attention_mask"].append(encoding["attention_mask"])
        tokenized_inputs["labels"].append(labels)

    return tokenized_inputs

# Tokenize datasets
print("Tokenizing training dataset...")
tokenized_train = train_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=train_dataset.column_names
)

print("Tokenizing validation dataset...")
tokenized_val = val_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=val_dataset.column_names
)

print("✅ Datasets tokenized")
print(f"   Training samples: {len(tokenized_train)}")
print(f"   Validation samples: {len(tokenized_val)}")
"""
tokens = ["Bật", "máy", "bơm", "ở", "khu", "A", "trong", "30", "phút"]
ner_tags = [0, 1, 2, 0, 3, 4, 0, 5, 6]

Từ:      Bật    máy    bơm    ở    khu    A    trong    30    phút
Tag:     O      B-DEV  I-DEV  O    B-AREA I-AREA O      B-DUR I-DUR
Số:      0      1      2      0    3      4      0      5     6

text = " ".join(tokens)
# text = "Bật máy bơm ở khu A trong 30 phút"

token_ids = [
    0,      # <s> (bắt đầu)
    8901,   # _Bật
    2345,   # _máy
    6789,   # _bơm
    1234,   # _ở
    5678,   # _khu
    9012,   # _A
    3456,   # _trong
    7890,   # _30
    4567,   # _phút
    2,      # </s> (kết thúc)
    1, 1, 1, ...  # <pad> (padding đến 128)
]

# Decode lại thành text:
decoded_tokens = [
    "<s>", "_Bật", "_máy", "_bơm", "_ở", "_khu", "_A", 
    "_trong", "_30", "_phút", "</s>", "<pad>", "<pad>", ...
]

labels = [-100] * 128  # 128 vị trí, tất cả là -100 (ignore)

word_idx = 0  # Bắt đầu từ từ đầu tiên

i = 0
token_id = 0
# Kiểm tra special token:
if token_id == tokenizer.bos_token_id:  # 0 == 0 → TRUE
    continue  # Bỏ qua
# labels[0] vẫn là -100

token_ids = [
    0,      # <s> (bắt đầu)
    8901,   # _Bật
    2345,   # _máy
    6789,   # _bơm
    1234,   # _ở
    5678,   # _khu
    9012,   # _A
    3456,   # _trong
    7890,   # _30
    4567,   # _phút
    2,      # </s> (kết thúc)
    1, 1, 1, ...  # <pad> (padding đến 128)
]

i = 1
token_id = 8901
word_idx = 0  # Đang xét từ "Bật"
# Decode token:
token_text = tokenizer.decode([8901]) = "_Bật"
token_clean = "Bật"
# Lấy từ hiện tại:
word = tokens[0] = "Bật"
# Kiểm tra khớp:
"Bật" in "Bật" → TRUE
# HOẶC
"Bật".startswith("Bật") → TRUE
# Gán nhãn:
labels[1] = ner_tags[0] = 0  # O
# Kiểm tra hết từ:
"Bật" == "Bật" → TRUE
word_idx = 1  # Chuyển sang từ "máy"

i = 2
token_id = 2345
word_idx = 1  # Đang xét từ "máy"
# Decode token:
token_clean = "máy"
# Lấy từ hiện tại:
word = tokens[1] = "máy"
# Kiểm tra khớp:
"máy" in "máy" → TRUE
# Gán nhãn:
labels[2] = ner_tags[1] = 1  # B-DEVICE
# Kiểm tra hết từ:
"máy" == "máy" → TRUE
word_idx = 2  # Chuyển sang từ "bơm"

OUTPUT cuối cùnG
tokenized_inputs = {
    "input_ids": [
        0, 8901, 2345, 6789, 1234, 5678, 9012, 3456, 7890, 4567, 2, 1, 1, ...
    ],
    "attention_mask": [
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, ...
    ],
    "labels": [
        -100, 0, 1, 2, 0, 3, 4, 0, 5, 6, -100, -100, -100, ...
    ]
}

Đặc điểm:

Mỗi token có 1 nhãn tương ứng
Special tokens và padding có nhãn -100
Sẵn sàng đưa vào model để training
"""

## 🎯 Step 8: Define Training Arguments

In [ ]:
"""
Step 8 (thay thế TrainingArguments)
Disable wandb logging
"""

from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
# batch = {
#     "input_ids": [
#         [0, 1234, 5678, 2, 1, 1, ...],      # Mẫu 1
#         [0, 9012, 3456, 7890, 2, 1, ...],   # Mẫu 2
#         [0, 2345, 6789, 2, 1, 1, ...]       # Mẫu 3
#     ],  # Shape: (3, 128)
    
#     "attention_mask": [
#         [1, 1, 1, 1, 0, 0, ...],
#         [1, 1, 1, 1, 1, 0, ...],
#         [1, 1, 1, 1, 0, 0, ...]
#     ],  # Shape: (3, 128)
    
#     "labels": [
#         [-100, 1, 2, -100, -100, ...],
#         [-100, 0, 3, 4, -100, ...],
#         [-100, 5, 6, -100, -100, ...]
#     ]  # Shape: (3, 128)
# }
import numpy as np
from seqeval.metrics import f1_score, precision_score, recall_score
import os

# Disable wandb
os.environ["WANDB_DISABLED"] = "true"

# Data collator
# Ghép nhiều sample lại thành 1 batch sao cho: input_ids, attention_mask, labels
# đều cùng chiều dài
data_collator = DataCollatorForTokenClassification(tokenizer)

# Metric computation
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    
    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    
    return {
        "precision": precision_score(true_labels, true_predictions),
        "recall": recall_score(true_labels, true_predictions),
        "f1": f1_score(true_labels, true_predictions),
    }

# Training arguments (with wandb disabled)
training_args = TrainingArguments(
    output_dir="./ner_model",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    push_to_hub=False,
    report_to="none",  # Disable all reporting (wandb, tensorboard, etc.)
)

print("✅ Training arguments configured")
print(f"   Epochs: {training_args.num_train_epochs}")
print(f"   Batch size: {training_args.per_device_train_batch_size}")
print(f"   Learning rate: {training_args.learning_rate}")
print(f"   Logging: Disabled (no wandb)")

## 🚀 Step 9: Train Model

In [ ]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Start training
print("\n🚀 Starting training...\n")
trainer.train()

print("\n✅ Training completed!")

## 📊 Step 10: Evaluate Model

In [ ]:
# Evaluate on validation set
results = trainer.evaluate()

print("\n📊 Evaluation Results:")
for key, value in results.items():
    print(f"  {key}: {value:.4f}")

## 💾 Step 11: Save Model

In [ ]:
# Save model and tokenizer
output_dir = "./ner_extractor_final"
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

# Copy label mapping
import shutil
shutil.copy('label_mapping.json', f'{output_dir}/label_mapping.json')

print(f"\n✅ Model saved to {output_dir}")
print("\n📦 Files to download:")
print("  - config.json")
print("  - pytorch_model.bin (or model.safetensors)")
print("  - label_mapping.json")

## 🧪 Step 12: Test Model

In [ ]:
# Test on new examples
test_examples = [
    "Bật tưới khu A trong 5 phút",
    "Độ ẩm ở khu B là bao nhiêu",
    "Chi phí tháng này",
    "Tắt đèn khu C",
    "Nhiệt độ khu 1 hiện tại",
    "Doanh thu quý 2",
    "Cách trồng cam sành"
]

def predict_entities(text):
    # Tokenize
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Predict
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=-1)[0]
    
    # Decode
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    labels = [id2label[p.item()] for p in predictions]
    
    # Extract entities
    entities = []
    current_entity = None
    
    for token, label in zip(tokens, labels):
        if token in ["<s>", "</s>", "<pad>"]:
            continue
            
        if label.startswith("B-"):
            if current_entity:
                entities.append(current_entity)
            current_entity = {"type": label[2:], "text": token.replace("_", " ").strip()}
        elif label.startswith("I-") and current_entity:
            current_entity["text"] += " " + token.replace("_", "").strip()
        elif label == "O" and current_entity:
            entities.append(current_entity)
            current_entity = None
    
    if current_entity:
        entities.append(current_entity)
    
    return entities

print("\n🧪 Testing model on new examples:\n")
for example in test_examples:
    entities = predict_entities(example)
    print(f"Text: {example}")
    print(f"Entities: {entities}")
    print()

## 📥 Step 13: Download Model Files

In [ ]:
# Zip model files for easy download
!zip -r ner_model.zip ner_extractor_final/
print("✅ Model zipped as ner_model.zip")
print("\n📥 Download ner_model.zip from Colab Files panel")
print("\n📋 Deployment instructions:")
print("1. Extract ner_model.zip")
print("2. Copy files to: C:\\Users\\ADMIN\\Desktop\\ex\\apps\\python-ai-service\\models\\ner_extractor\\")
print("3. Restart Python AI service")